In [21]:
#Importing useful packages
import sqlite3
import csv
import pandas as pd
from datetime import datetime

#Returns a Dataframe containing a sample of size "number" from the specified month and year
def get_sample(year,month,number):
    #Error Checking
    if number < 0:
        print("Bad Number Value\n")
        return
    if year < 2012 | year > 2014:
        print("Bad year value\n")
        return
    
    #Change to directory of your database!
    conn = sqlite3.connect(r'E:\Summer Project\yellow_taxi_'+str(year)+'.sqlite')
    c = conn.cursor()
    #For small samples (less than 10000) adding them this way is fastest
    if number<10000:
        sample = pd.read_sql_query('select * from '+str(month)+
                                   ' where rowid = (abs(random()) % (select (select max(rowid) from '+str(month)+')+1))',conn)
        for i in range(number-1):
            sample = sample.append(pd.read_sql_query('select * from '+str(month)+
                                                     ' where rowid = (abs(random()) % (select (select max(rowid) from '+str(month)+')+1))',conn))#row,ignore_index=True)
    else:
        #This is the fastest way to get a random sample for large entries
        sample = pd.read_sql_query('select * from '+str(month)+' order by random() limit '+str(number),conn)
    
    #Turns the dropoff_datetime column into an array of time values
    tmpdate = sample['dropoff_datetime'].get_values()
    pickuptmp = sample['pickup_datetime'].get_values()
    
    #Initiliazing three arrays that will be our new columns in our sample
    dropoff_hour = [0]*(number)
    dropoff_day = [0]*(number)
    day_of_week = [0]*(number)
    time_in_taxi = [0]*(number)
    
        
    #Loop that will strip the time from every entry in tmpdate and assign it to the right column
    for i in range(number):
        tmp = datetime.strptime(tmpdate[i], '%Y-%m-%d %H:%M:%S')
        tmp2 = datetime.strptime(pickuptmp[i],'%Y-%m-%d %H:%M:%S')
        #print("Times:" +str(tmp)+" - "+ str(tmp2))
        #print(tmp-tmp2)
        dropoff_hour[i] = tmp.hour
        dropoff_day[i] = tmp.day
        day_of_week[i] = tmp.weekday()
        tmp3 = tmp-tmp2
        time_in_taxi[i] = (tmp3.seconds//60)# + tmp3.hour*60 
    
    #Creating the new columns for our sample
    sample['dropoff_hour'] = dropoff_hour
    sample['dropoff_day'] = dropoff_day
    sample['weekday'] = day_of_week
    sample['time_in_taxi'] = time_in_taxi
    
    #Returning the sample
    return sample

In [33]:
#Example of get_sample
x = get_sample(2013,"january",20)
print(x)
#This will print out the sample to a csv file so it can be loaded into JMP or other software
x.to_csv('test_sample3.csv')


  vendor_id      pickup_datetime     dropoff_datetime  passenger_count  \
0       CMT  2013-01-11 05:08:09  2013-01-11 05:09:03                1   
0       CMT  2013-01-14 07:29:01  2013-01-14 07:40:21                1   
0       VTS  2013-01-27 12:13:00  2013-01-27 12:20:00                4   
0       CMT  2013-01-15 08:42:03  2013-01-15 08:42:06                1   
0       CMT  2013-01-09 05:37:33  2013-01-09 05:42:41                1   
0       VTS  2013-01-29 11:26:00  2013-01-29 11:34:00                5   
0       CMT  2013-01-08 01:20:50  2013-01-08 01:30:21                1   
0       VTS  2013-01-18 18:02:00  2013-01-18 18:32:00                1   
0       VTS  2013-01-21 06:52:00  2013-01-21 06:56:00                1   
0       VTS  2013-01-05 04:17:00  2013-01-05 04:48:00                1   
0       CMT  2013-01-19 01:21:18  2013-01-19 01:28:09                1   
0       CMT  2013-01-15 09:34:49  2013-01-15 09:47:43                1   
0       CMT  2013-01-18 22:13:55  2013